In [16]:
import numpy as np
import torch
data = torch.randn(100, 7, 3)
last_n_dims = 2
dim = np.prod(data.shape[-last_n_dims:])
data = data.reshape([-1, dim])
data.shape
data.min(axis = 0)

torch.return_types.min(
values=tensor([-2.7490, -2.4052, -2.0001, -2.4288, -2.1819, -2.1208, -2.4951, -3.1513,
        -2.4003, -2.5508, -4.2990, -2.2741, -2.9015, -2.0374, -2.5577, -2.4517,
        -2.7667, -2.0345, -2.4988, -2.2729, -2.4419]),
indices=tensor([36, 35, 40, 14, 75, 57,  4, 92, 34, 43, 72, 20, 24, 49, 45, 31, 28, 68,
         4, 92, 15]))

In [35]:
from typing import Union
import zarr
def _fit(data: Union[torch.Tensor, np.ndarray, zarr.Array],
             last_n_dims = 1,
             dtype = torch.float32,
             mode = 'limits',
             output_max = 1, 
             output_min = -1, 
             range_eps = 1e-4,
             fit_offset = True):

        assert mode in ['limits', 'gaussian']
        assert last_n_dims > 0
        assert output_max > output_min
        
        if isinstance(data, zarr.Array):
            data = data[:]
        if isinstance(data, np.ndarray):
            data = torch.from_numpy(data)
        if dtype is not None:
            data = data.type(dtype)
            
        dim = 1 
        if last_n_dims > 0:
            dim = np.prod(data.shape[-last_n_dims:])
        data = data.reshape(-1, dim)
        
        input_min, _ = data.min(axis=0)
        input_max, _ = data.max(axis=1)
        input_mean = data.mean(axis=0)
        input_std = data.std(axis =0)
        
        if mode == 'limits':
            if fit_offset:
                input_range = input_max - input_min
                output_range = output_max - output_min 
                ignore_dim = input_range < range_eps
                input_range[ignore_dim] = output_range
                 
                scale = output_range / input_range
                offset = output_min - input_min * scale
                print(input_min[ignore_dim])
                offset[ignore_dim] = (output_max + output_min) / 2 - input_min[ignore_dim]
                
                print(offset)
                x= data
                x= x*scale+ offset
                print(x)
data = torch.Tensor([[100,1],[100.00001,100]])
_fit(data)
x = data.reshape(-1, 2)


tensor([100.])
tensor([-100.0000,   -1.0202])
tensor([[ 0.0000e+00, -1.0000e+00],
        [ 7.6294e-06,  1.0000e+00]])
